# Parse all PE to transform JSON files

In [6]:
from peanalysis.parse import PEParser
import os
import json
import numpy as np
from IPython.display import display
from ipywidgets import widgets

## Loads all path of pe files

In [9]:
input_text=widgets.Text()
all_file = []
path_repo = '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/'




In [3]:
for root, subFolders, files in os.walk(path_repo):
    for file in files:
        path_file = os.path.join(root, file)
        all_file.append(path_file)

In [4]:
all_file

['/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/win32.exe',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/TrojanWin32.Duqu.Stuxnet.zip',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/TrojanWin32.Duqu.Stuxnet.md5',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/TrojanWin32.Duqu.Stuxnet.pass',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/TrojanWin32.Duqu.Stuxnet.sha256',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Win32.Turla/decrypted_rkctl_Win32.dll',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Win32.Turla/decrypted_inj_snake_x64.dll',
 '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Win32.Turla/Win32.Turla.md5',
 '/home/slarinier/Documents/projet/viralstudio/

##  Store all PEs parsed

In [5]:
all_pe = []
for p in all_file:
    try:
        pe = PEParser(p)
        pe.load()
        pe.dump_sections()
        pe.dump_imports()
        pe.dump_exports()
        pe.dump_resources()
        pe.dump_tls()
        all_pe.append(pe)
    except:
        pass

ordinal not found MFC42.DLL 5076
ordinal not found MFC42.DLL 4341
ordinal not found MFC42.DLL 4349
ordinal not found MFC42.DLL 4890
ordinal not found MFC42.DLL 4531
ordinal not found MFC42.DLL 4545
ordinal not found MFC42.DLL 4543
ordinal not found MFC42.DLL 4526
ordinal not found MFC42.DLL 4529
ordinal not found MFC42.DLL 4524
ordinal not found MFC42.DLL 4964
ordinal not found MFC42.DLL 4961
ordinal not found MFC42.DLL 4108
ordinal not found MFC42.DLL 6055
ordinal not found MFC42.DLL 4078
ordinal not found MFC42.DLL 1776
ordinal not found MFC42.DLL 4407
ordinal not found MFC42.DLL 5240
ordinal not found MFC42.DLL 2385
ordinal not found MFC42.DLL 5163
ordinal not found MFC42.DLL 6374
ordinal not found MFC42.DLL 4353
ordinal not found MFC42.DLL 5290
ordinal not found MFC42.DLL 3798
ordinal not found MFC42.DLL 4837
ordinal not found MFC42.DLL 4441
ordinal not found MFC42.DLL 2648
ordinal not found MFC42.DLL 2055
ordinal not found MFC42.DLL 6376
ordinal not found MFC42.DLL 3748
ordinal no

ordinal not found COMCTL32.DLL 413
ordinal not found COMCTL32.DLL 410
ordinal not found SHLWAPI.DLL 12
ordinal not found COMCTL32.DLL 17
ordinal not found SHLWAPI.DLL 157
ordinal not found MSVBVM60.DLL 582
ordinal not found MSVBVM60.DLL 583
ordinal not found MSVBVM60.DLL 585
ordinal not found MSVBVM60.DLL 187
ordinal not found MSVBVM60.DLL 588
ordinal not found MSVBVM60.DLL 516
ordinal not found MSVBVM60.DLL 517
ordinal not found MSVBVM60.DLL 518
ordinal not found MSVBVM60.DLL 660
ordinal not found MSVBVM60.DLL 669
ordinal not found MSVBVM60.DLL 593
ordinal not found MSVBVM60.DLL 300
ordinal not found MSVBVM60.DLL 301
ordinal not found MSVBVM60.DLL 595
ordinal not found MSVBVM60.DLL 303
ordinal not found MSVBVM60.DLL 598
ordinal not found MSVBVM60.DLL 305
ordinal not found MSVBVM60.DLL 306
ordinal not found MSVBVM60.DLL 520
ordinal not found MSVBVM60.DLL 307
ordinal not found MSVBVM60.DLL 309
ordinal not found MSVBVM60.DLL 709
ordinal not found MSVBVM60.DLL 631
ordinal not found MSVBVM

ordinal not found COMCTL32.DLL 17
ordinal not found SHELL32.DLL 680
ordinal not found COMCTL32.DLL 17
ordinal not found SHLWAPI.DLL 12
ordinal not found MSI.DLL 141
ordinal not found MSI.DLL 168
ordinal not found MSI.DLL 160
ordinal not found MSI.DLL 158
ordinal not found MSI.DLL 115
ordinal not found MSI.DLL 159
ordinal not found MSI.DLL 117
ordinal not found MSI.DLL 8
ordinal not found MSI.DLL 44
ordinal not found MSI.DLL 204
ordinal not found MSI.DLL 189
ordinal not found MSI.DLL 67
ordinal not found MSI.DLL 31
ordinal not found MSI.DLL 137
ordinal not found MSI.DLL 91
ordinal not found COMCTL32.DLL 17
ordinal not found MFC42.DLL 4234
ordinal not found MFC42.DLL 2648
ordinal not found MFC42.DLL 641
ordinal not found MFC42.DLL 324
ordinal not found MFC42.DLL 3597
ordinal not found MFC42.DLL 4425
ordinal not found MFC42.DLL 5280
ordinal not found MFC42.DLL 1775
ordinal not found MFC42.DLL 6052
ordinal not found MFC42.DLL 2514
ordinal not found MFC42.DLL 4710
ordinal not found MFC42.DL

ordinal not found MSVBVM60.DLL 588
ordinal not found MSVBVM60.DLL 516
ordinal not found MSVBVM60.DLL 631
ordinal not found MSVBVM60.DLL 525
ordinal not found MSVBVM60.DLL 632
ordinal not found MSVBVM60.DLL 563
ordinal not found MSVBVM60.DLL 717
ordinal not found MSVBVM60.DLL 570
ordinal not found MSVBVM60.DLL 100
ordinal not found COMCTL32.DLL 321
ordinal not found COMCTL32.DLL 13
ordinal not found COMCTL32.DLL 320
ordinal not found COMCTL32.DLL 325
ordinal not found COMCTL32.DLL 322
ordinal not found COMCTL32.DLL 321
ordinal not found COMCTL32.DLL 322
ordinal not found COMCTL32.DLL 320
ordinal not found COMCTL32.DLL 13
ordinal not found COMCTL32.DLL 325
ordinal not found COMCTL32.DLL 321
ordinal not found COMCTL32.DLL 322
ordinal not found COMCTL32.DLL 325
ordinal not found COMCTL32.DLL 320
ordinal not found COMCTL32.DLL 13
ordinal not found COMCTL32.DLL 17
ordinal not found COMCTL32.DLL 17
ordinal not found COMCTL32.DLL 13
ordinal not found COMCTL32.DLL 322
ordinal not found COMCTL32

In [6]:
len(all_pe)

767

In [7]:
all_pe[0].dict_pe

{'Date Compilation': '2011-10-17 21:06:28',
 'address_entrypoint': '0x10570',
 'assembly': '',
 'exports': OrderedDict(),
 'hashes': {'impfuzzy': '24:zPst7seuTApjtKmwaVIK9Ygk/ixKaVJ9qgqkqEqlwE1u3zg:AsefRHwHp/8KaVJ9qgqkqEqjog',
  'imphash': 'c4b26918bdf8111b995d424e073d2d2a',
  'md5': 'c9a31ea148232b201fe7cb7db5c75f5e',
  'sha1': 'b3074b26b346cb76605171ba19616baf821acf66',
  'sha256': '9d88425e266b3a74045186837fbd71de657b47d11efefcf8b3cd185a884b5306',
  'ssdeep': '384:bJu/osVhICBqnHH1vZGHvCzQ3T022+u/IlCq7HuekK4:lw/rBQnVgHvqQ392//MRkK4'},
 'imports': OrderedDict([('NTOSKRNL.EXE',
               ['IoDeleteDevice',
                'IoFreeWorkItem',
                'MmUnmapIoSpace',
                'MmGetPhysicalAddress',
                'ExAllocatePool',
                'IoAllocateWorkItem',
                'MmMapIoSpace',
                'IoAttachDeviceToDeviceStack',
                'IoCreateSymbolicLink',
                'IoInitializeRemoveLockEx',
                'IoCreateDevice',
    

## Record in json files the data parsed

In [8]:
for p in all_pe:
    json.dump( p.dict_pe,open('%s.json' %os.path.join('jsonfiles', p.dict_pe['hashes']['sha256']),'w'))

In [9]:
len(all_pe)

767

## Check Collisions

In [10]:
all_sha256 = [ p.dict_pe['hashes']['sha256']for p in all_pe]

In [11]:
len(all_sha256)

767

In [12]:
len(set(all_sha256))

724

In [13]:
from collections import Counter
c = Counter()
for s in all_sha256:
    c[s] +=1

In [14]:
collision = [ k for k,v in c.items() if v > 1]
collision

['8bb5c766de0a73dc0eff7c9fce086565b6220465185e258c21c5b9dfb0bef51d',
 '426511145595346a6aee1d3483685ad32674f626a4695bb91aa82c1b016a0f1c',
 '1e55abb94951cedc548fd8d67bd1b50476808f1d0ae72f9842181761ff92f83f',
 '9c17f267f79597ee01515f5ef925375d8a19844830cc46917a3d1b5bcb0ba4c3',
 '7249b1a5082c9d9654d9fac3bb5e965ea23e395554d3351b77dd4f29677426fe',
 '3639e8cc463922b427ea20dce8f237c0c0e82aa51d2502c48662e60fb405f677',
 'a3667153a6322fb8d4cf8869c094a05e995e2954fda833fe14304837ed4fd0bd',
 'a6ff8dfe654da70390cd71626cdca8a6f6a0d7980cd7d82269373737b04fd206',
 'b670fe2d803705f811b5a0c9e69ccfec3a6c3a31cfd42a30d9e8902af7b9ed80',
 '0cfc34fa76228b1afc7ce63e284a23ce1cd2927e6159b9dea9702ad9cb2a6300',
 '16166533c69f2f04110e8b8e9cc45ed2aeaf7850fa68845c64d92ff907dd44f0',
 '1b0eb1a1591140175d1ac111a98c89472b196599baf13ef67ee7f63d0052b00e',
 '19e818d0da361c4feedd456fca63d68d4b024fbbd3d9265f606076c7ee72e8f8',
 '003315b0aea2fcb9f77d29223dd8947d0e6792b3a0227e054be8eb2a11f443d9',
 '6b91fdb0992ca029c913092db7b4fd94

In [15]:
[(p.dict_pe['hashes']['sha256'] ,p.dict_pe['path']) for p in all_pe if p.dict_pe['hashes']['sha256'] in collision]

[('9d88425e266b3a74045186837fbd71de657b47d11efefcf8b3cd185a884b5306',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/TrojanWin32.Duqu.Stuxnet/win32.exe'),
 ('1e55abb94951cedc548fd8d67bd1b50476808f1d0ae72f9842181761ff92f83f',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/EquationGroup.DoubleFantasy/DoubleFantasy_2A12630FF976BA0994143CA93FECD17F'),
 ('ef32516eb5658c65299cb1c9a0f7ec552216f4b9d2975d074ff31eacb3003a19',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Surtr/hV46VA.dll'),
 ('ef32516eb5658c65299cb1c9a0f7ec552216f4b9d2975d074ff31eacb3003a19',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Surtr/ydrHrp_One.dll'),
 ('ef32516eb5658c65299cb1c9a0f7ec552216f4b9d2975d074ff31eacb3003a19',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/Surtr/2.dll'),
 ('df4bbd02dcd8b8b9e1374c6f71f2e2da8518d39337b35983874266e8fff055e1',
  '/home/slarinier/Documents/projet/vira

In [16]:
sorted(_, key = lambda x : x[0])

[('003315b0aea2fcb9f77d29223dd8947d0e6792b3a0227e054be8eb2a11f443d9',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/EquationGroup/Equation_KasperskyReport_and_Additionalsamples/Fanny_0A209AC0DE4AC033F31D6BA9191A8F7A'),
 ('003315b0aea2fcb9f77d29223dd8947d0e6792b3a0227e054be8eb2a11f443d9',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/EquationGroup.Fanny/Fanny_0A209AC0DE4AC033F31D6BA9191A8F7A'),
 ('01259a104a0199b794b0c61fcfc657eb766b2caeae68d5c6b164a53a97874257',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/WMIGhost/01259a104a0199b794b0c61fcfc657eb766b2caeae68d5c6b164a53a97874257'),
 ('01259a104a0199b794b0c61fcfc657eb766b2caeae68d5c6b164a53a97874257',
  '/home/slarinier/Documents/projet/viralstudio/theZoo/malwares/Binaries/WMIGhost/WMIGhost/01259a104a0199b794b0c61fcfc657eb766b2caeae68d5c6b164a53a97874257'),
 ('05455efecab4a7931fa53a3c2008d04fc6b539c5e8f451f19b617bd9b3ebcd83',
  '/home/slarinier/Documents/p

# First vector of features

## Storage metadata in redis

In [2]:
import glob
from redis_management import RedisManagement
from collections import OrderedDict
from ml_helpers import ml_helpers
rmgt = RedisManagement('malwares')

In [2]:
features = ['size of file', 'number of sections', 'median of entropy', 'number of imports', 'number of exports']

In [7]:
all_json_files = [ json.load(open(path))for path in glob.glob('jsonfiles/*')]

In [1]:
for j in all_json_files:
    malware = OrderedDict()
    malware[features[0]] = j['size']
    malware[features[1]] = j['number_sections']
    malware[features[2]] = sum( sec['entropy'] for sec in j['sections']) / j['number_sections']
    malware[features[3]] = sum(len(v) for k,v in j['imports'].items())
    malware[features[4]] =  sum(len(v) for k,v in j['exports'].items())
    rmgt.record_vector(j['hashes']['sha256'],'first_vector',malware)
    rmgt.record_malware_label(j['hashes']['sha256'],j['path'],path_repo)   

NameError: name 'all_json_files' is not defined

## Create Matrix of vectors features

In [11]:
ml_h = ml_helpers(rmgt.redis_client)
mat = ml_h.create_matrix('first_vector')
mat

array([[  1.06496000e+05,   6.00000000e+00,   3.22704696e+00,
          9.50000000e+01,   7.00000000e+00],
       [  1.84320000e+05,   5.00000000e+00,   5.12440937e+00,
          1.31000000e+02,   0.00000000e+00],
       [  1.84320000e+05,   5.00000000e+00,   5.20316476e+00,
          1.31000000e+02,   0.00000000e+00],
       ..., 
       [  1.32612000e+05,   6.00000000e+00,   4.63970869e+00,
          2.17000000e+02,   6.00000000e+00],
       [  9.52320000e+04,   3.00000000e+00,   5.23801030e+00,
          4.60000000e+01,   0.00000000e+00],
       [  1.84320000e+05,   5.00000000e+00,   5.12441068e+00,
          1.31000000e+02,   0.00000000e+00]])

In [16]:
mat.shape

(724, 5)

In [17]:
np.save('matrix_first_vector', mat)